In [ ]:
!pip install azure-eventhub

In [2]:
import pandas as pd
import random
from datetime import datetime
from pyspark.sql import SparkSession                    
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
import pandas as pd
import random
from datetime import datetime
import pytz

StatementMeta(, ee664ce5-2438-4967-b99f-7c91a493ed27, 4, Finished, Available, Finished)

In [ ]:


# Define device to store/location mapping
store_mapping = {
    (1, 10): ("Miami", "S001"),
    (11, 20): ("San Diego", "S002"),
    (21, 30): ("New York", "S003"),
    (31, 40): ("San Francisco", "S004"),
    (41, 50): ("Las Vegas", "S005"),
    (51, 60): ("London", "S006"),
    (61, 70): ("Dubai", "S007"),
    (71, 80): ("Singapore", "S008"),
    (81, 90): ("Tokyo", "S009"),
    (91, 100): ("Chicago", "S010"),
}

# Get current UTC time
now_utc = datetime.utcnow()

# Generate 100 records
records = []
for i in range(1, 101):
    device_id = f"TH{i:03}"
    # Determine store and city
    for (start, end), (city, store) in store_mapping.items():
        if start <= i <= end:
            city_name = city
            store_id = store
            break
    # Set values
    battery = random.randint(0, 100) 
    temp = round(random.uniform(60.0, 74.0), 1)
    temp_uom = "F"
    enqueued_time = now_utc
    event_processed_time = now_utc
    event_enqueued_time = now_utc
    partition_id = random.randint(1, 4)

    records.append({
        "DeviceId": device_id,
        "City": city_name,
        "StoreID": store_id,
        "EnqueuedTimeUTC": enqueued_time,
        "BatteryLevel": battery,
        "Temp": temp,
        "Temp_UoM": temp_uom
    })

# Create DataFrame
df = pd.DataFrame(records)

# Show first few rows
print(df.head())

# Optionally save to CSV
# df.to_csv("thermostat_realtime_data.csv", index=False)


In [4]:


# Create Pandas DataFrame
df = pd.DataFrame(records)

# Convert to Spark DataFrame
spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("DeviceId", StringType(), True),
    StructField("City", StringType(), True),
    StructField("StoreID", StringType(), True),
    StructField("EnqueuedTimeUTC", TimestampType(), True),
    StructField("BatteryLevel", IntegerType(), True),
    StructField("Temp", FloatType(), True),
    StructField("Temp_UoM", StringType(), True)
])

spark_df = spark.createDataFrame(df, schema=schema)

# Write to Lakehouse table
#spark_df.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("ThermostatRealtime")
#spark_df.write.mode("append").saveAsTable("Thermostat")


StatementMeta(, ee664ce5-2438-4967-b99f-7c91a493ed27, 6, Finished, Available, Finished)

In [5]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime

# Custom JSON encoder to handle datetime
class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # Convert datetime to string
        return super().default(obj)

# Event Hub details
CONNECTION_STR = "#Connection string-primary key#"
EVENT_HUB_NAME = "#Event hub name#"


# Initialize Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STR, eventhub_name=EVENT_HUB_NAME)

event_data_batch = producer.create_batch()
for row in spark_df.collect():
    record = row.asDict()
    json_payload = json.dumps(record, cls=DateTimeEncoder)  # ✅ Handles datetime
    event_data_batch.add(EventData(json_payload))

# Send the batch
producer.send_batch(event_data_batch)
producer.close()

print("✅ Data sent to Eventstream endpoint.")


StatementMeta(, ee664ce5-2438-4967-b99f-7c91a493ed27, 7, Finished, Available, Finished)

✅ Data sent to Eventstream endpoint.
